In [1]:
import pandas as pd


import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime

In [6]:
path = '/data/public/MLA/VCHAMPS-Test/procedures_test.csv'
procedures = pd.read_csv(path,engine = 'python',on_bad_lines='warn')
procedures.head()

Skipping line 18710187: unexpected end of data


,Unnamed: 0,Internalpatientid,Age at procedure,Procedure date,Procedure code,Procedure code description,State
0,112,100,52.559428,2014-04-19 11:18:05.0,99343,HOME VISIT FOR THE EVALUATION AND MANAGEMENT O...,New York
1,113,100,52.559428,2014-04-19 11:18:05.0,G0154,DIRECT SKILLED NURSING SERVICES OF A LICENSED ...,New York
2,114,100,52.753358,2014-06-29 08:27:24.0,97003,OCCUPATIONAL THERAPY EVALUATION,New York
3,115,100,52.753358,2014-06-29 08:27:24.0,97535,"SELF-CARE/HOME MANAGEMENT TRAINING (EG, ACTIVI...",New York
4,116,100,53.094739,2014-11-01 03:03:17.0,G0155,SERVICES OF CLINICAL SOCIAL WORKER IN HOME HEA...,New York


In [7]:
procedures['Procedure code description'].nunique()

18637

In [9]:
procedures[procedures.duplicated(subset = ['Internalpatientid', 'Procedure date','Procedure code description'],keep = False)]

,Unnamed: 0,Internalpatientid,Age at procedure,Procedure date,Procedure code,Procedure code description,State
17,129,100,55.094130,2016-10-31 21:42:18.0,99233,"SUBSEQUENT HOSPITAL CARE, PER DAY, FOR THE EVA...",New York
18,130,100,55.094130,2016-10-31 21:42:18.0,99233,"SUBSEQUENT HOSPITAL CARE, PER DAY, FOR THE EVA...",New York
19,131,100,55.103925,2016-11-04 11:38:04.0,74000,"RADIOLOGIC EXAMINATION, ABDOMEN; SINGLE ANTERO...",New York
20,132,100,55.103925,2016-11-04 11:38:04.0,74000,"RADIOLOGIC EXAMINATION, ABDOMEN; SINGLE ANTERO...",New York
21,133,100,55.103925,2016-11-04 11:38:04.0,71010,"RADIOLOGIC EXAMINATION, CHEST; SINGLE VIEW, FR...",New York
...,...,...,...,...,...,...,...
18710077,89978293,64272,84.520915,2009-08-13 09:28:01.0,99213,OFFICE OR OTHER OUTPATIENT VISIT FOR THE EVALU...,Connecticut
18710162,89979159,64636,67.238708,2005-12-21 17:56:33.0,82043,"ALBUMIN; URINE (EG, MICROALBUMIN), QUANTITATIVE",Arkansas
18710166,89979163,64636,67.238708,2005-12-21 17:56:33.0,82043,"ALBUMIN; URINE (EG, MICROALBUMIN), QUANTITATIVE",Arkansas
18710178,89979358,64640,77.234599,2008-01-11 05:54:07.0,90658,"INFLUENZA VIRUS VACCINE, TRIVALENT (IIV3), SPL...",New Jersey


In [10]:
procedures = procedures.drop_duplicates(subset = ['Internalpatientid', 'Procedure date','Procedure code description'],keep = 'first')

### Add New features: 
Number of procedures

Number of surgery procedures

Minimum number of surgery procedures

Maximum number of surgery procedures

In [11]:
surgery_pro = procedures.loc[procedures['Procedure code description'].str.contains('SURG')]
surgery_pro['num_surgery_pro'] = surgery_pro['Procedure code'].groupby(surgery_pro['Internalpatientid']).transform('count')

/tmp/ipykernel_121237/3717331933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  surgery_pro['num_surgery_pro'] = surgery_pro['Procedure code'].groupby(surgery_pro['Internalpatientid']).transform('count')


In [12]:
procedures['total_procedure'] = procedures['Procedure code description'].groupby(procedures['Internalpatientid']).transform('count')

In [13]:
columns = ['Internalpatientid', 'total_procedure']
df = procedures[columns]
df = df.groupby('Internalpatientid').mean().reset_index(drop = False)
df = df.merge(surgery_pro[['Internalpatientid','num_surgery_pro']], how = 'left',on = 'Internalpatientid')
df = df.groupby('Internalpatientid').mean().reset_index(drop = False)
df['num_surgery_pro'] = df['num_surgery_pro'].fillna(0)
df

,Internalpatientid,total_procedure,num_surgery_pro
0,6,767.0,3.0
1,12,247.0,0.0
2,17,338.0,1.0
3,22,259.0,0.0
4,25,881.0,9.0
...,...,...,...
26889,169037,783.0,0.0
26890,169045,40.0,0.0
26891,169058,86.0,1.0
26892,169059,520.0,0.0


In [15]:
#df.to_csv('/home/daisy/FDA_Dataset/procedures_test_clean.csv')